In [3]:
# Import libraries required in general
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen
import json
import objectpath
import pandas as pd
import numpy as np

In [46]:
# Import libraries for scraping TimesHigherEducation

from selenium import webdriver
from selenium.webdriver.chrome.service import Service

# Set Chrome webdriver location
webdriver_location = r'C:\Users\Lenovo V15\Downloads\chromedriver_win32.exe'

# Create a Service object with the webdriver_location
service = Service(executable_path=webdriver_location)

# Create two webdriver objects, one for each of the two addresses
# Pass the Service object to the webdriver.Chrome constructor

stats_browser = webdriver.Chrome(service=service)
scores_browser = webdriver.Chrome(service=service)


# The URL here select all of the university rankings in one go, so that next pages do not have to be 'clicked'
url_stats = 'https://www.timeshighereducation.com/world-university-rankings/2020/world-ranking#!/page/0/length/-1/sort_by/rank/sort_order/asc/cols/stats'
url_scores = 'https://www.timeshighereducation.com/world-university-rankings/2020/world-ranking#!/page/0/length/-1/sort_by/rank/sort_order/asc/cols/scores'


# Use the webdriver to request the ranking webpage
stats_browser.get(url_stats)
scores_browser.get(url_scores)

# Collect the webpage HTML after its loading
stats_page_html = stats_browser.page_source
scores_page_html = scores_browser.page_source

# Parse the HTML using BeautifulSoup
stats_page_soup = soup(stats_page_html, 'html.parser')
scores_page_soup = soup(scores_page_html, 'html.parser')


# Close the stats_browser
stats_browser.close()
# Close the scores_browser
scores_browser.close()


In [47]:
# Collect data/HTML objects for the stats page
rank_obj = stats_page_soup.findAll("td", {"class": "rank sorting_1 sorting_2"})
names_obj = stats_page_soup.findAll("td", {"class": "name namesearch"})
stats_number_students_obj = stats_page_soup.findAll("td", {"class": "stats stats_number_students"})
stats_student_staff_ratio_obj = stats_page_soup.findAll("td", {"class": "stats stats_student_staff_ratio"})
stats_pc_intl_students_obj = stats_page_soup.findAll("td", {"class": "stats stats_pc_intl_students"})
stats_female_male_ratio_obj = stats_page_soup.findAll("td", {"class": "stats stats_female_male_ratio"})


# Repeat for the scores page
overall_score_obj = scores_page_soup.findAll("td", {"class": "scores overall-score"})
teaching_score_obj = scores_page_soup.findAll("td", {"class": "scores teaching-score"})
research_score_obj = scores_page_soup.findAll("td", {"class": "scores research-score"})
citations_score_obj = scores_page_soup.findAll("td", {"class": "scores citations-score"})
industry_income_score_obj = scores_page_soup.findAll("td", {"class": "scores industry_income-score"})
international_outlook_score_obj = scores_page_soup.findAll("td", {"class": "scores international_outlook-score"})


# Initialize empty lists to store the data
rank, names, number_students, student_staff_ratio, intl_students, female_male_ratio = [], [], [], [], [], []
overall_score, teaching_score, research_score, citations_score, industry_income_score, international_outlook_score = [], [], [], [], [], []


# Extract data from HTML objects and append to the respective lists
for i in range(len(names_obj)):
    try: 
        name = names_obj[i].a.get('href')
    except AttributeError:
        name = ""
    names.append(name)
    
    rank.append(rank_obj[i].text)
    number_students.append(stats_number_students_obj[i].text)
    student_staff_ratio.append(stats_student_staff_ratio_obj[i].text)
    intl_students.append(stats_pc_intl_students_obj[i].text)
    female_male_ratio.append(stats_female_male_ratio_obj[i].text[:2])

    overall_score.append(overall_score_obj[i].text)
    teaching_score.append(teaching_score_obj[i].text)
    research_score.append(research_score_obj[i].text)
    citations_score.append(citations_score_obj[i].text)
    industry_income_score.append(industry_income_score_obj[i].text)
    international_outlook_score.append(international_outlook_score_obj[i].text)

In [ ]:
# Now, collect the information on the location of each University

from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

web_address =[]

for i in range(len(names)):
    try: 
        name = names_obj[i].a.get('href')
    except AttributeError:
        name = ""
    web_address.append('https://www.timeshighereducation.com' + name)


# Initialize empty lists to store address information
full_address_list, streetAddress_list, addressLocality_list, addressRegion_list, postalCode_list, addressCountry_list = [], [], [], [], [], []


driver = webdriver.Chrome(service=service)

for web in web_address:
    driver.get(web)  # Load the web page

    # Extract address information using Selenium
    try:
        full_address_element = driver.find_element(By.CSS_SELECTOR, ".institution-info__contact-detail.institution-info__contact-detail--address")
        full_address = full_address_element.text.strip()
    except NoSuchElementException:
        full_address = 'N/A'  # Replace 'N/A' with a default value of your choice

    full_address_list.append(full_address)
    
    # print out to confirm it works
    # print('{} out of {}'.format(len(full_address_list), len(web_address)), full_address)

# Close the browser
driver.quit()


In [ ]:
# Create a DataFrame to store the collected data
df = pd.DataFrame({
    'rank': rank,
    'name': names,
    'number_students': number_students,
    'student_staff_ratio': student_staff_ratio,
    'intl_students': intl_students,
    'female_male_ratio': female_male_ratio,
    'overall_score': overall_score,
    'teaching_score': teaching_score,
    'research_score': research_score,
    'citations_score': citations_score,
    'industry_income_score': industry_income_score,
    'international_outlook_score': international_outlook_score,
    'address': full_address_list
})


df.to_csv('uni_02.csv', encoding='utf-8', index=False)

In [53]:
print(web_address[1000], full_address_list[1000])

https://www.timeshighereducation.com/world-university-rankings/university-zaragoza Calle de Pedro Cerbuna 12, Zaragoza, 50009, Spain
